In [3]:
import os, sys; sys.path.append("../utils")
from constants import *

import numpy as np
import pandas as pd

from sklearn.metrics import cohen_kappa_score as kappa

import torch as t
from sklearn.linear_model import LogisticRegression

from tqdm import trange


aspects = ["consistency", "coherence", "fluency", "relevance"]

In [10]:
split = 0.7

for model in ["mistral", "llama2", "llama3"]:
    results = pd.DataFrame(columns=["dataset"]+aspects)
    for dataset in ["summeval"]:
        row = [dataset]
        path = f"{gdrive_path}/prompts_short/{model}/{dataset}_mine_1.jsonl"
        ref = pd.read_json(path, orient="records", lines=True)
        for aspect in aspects:
            c1 = t.load(f"{gdrive_path}/activations_short/{model}/{dataset}_{aspect}_1.pt")
            c2 = t.load(f"{gdrive_path}/activations_short/{model}/{dataset}_{aspect}_2.pt")
            X = c1 - c2
            y = ref[aspect].apply(lambda x: (x[0] > x[1])*1).values
            split_ix = int(split*len(X))
            accs, ks = [], []
            for _ in trange(10, desc="fitting models"):
                perm = t.randperm(len(X))
                _X, _y = X[perm], y[perm]
                X_train, X_test = X[:split_ix], X[split_ix:]
                y_train, y_test = y[:split_ix], y[split_ix:]
                lr = LogisticRegression(max_iter=10_000)
                lr.fit(X_train, y_train)
                preds = lr.predict(X_test)
                acc = (preds == y_test).mean()
                k = kappa(preds, y_test)
                accs.append(acc)
                ks.append(k)
            row.extend([np.mean(accs)])
        results.loc[len(results)] = row
    results.set_index("dataset", inplace=True)
    print(model)
    display(results)

fitting models: 100%|██████████| 10/10 [10:01<00:00, 60.13s/it]


mistral


,consistency,coherence,fluency,relevance
dataset,,,,
summeval,0.785833,0.641111,0.704722,0.589444


fitting models: 100%|██████████| 10/10 [05:08<00:00, 30.90s/it]

llama2


,consistency,coherence,fluency,relevance
dataset,,,,
summeval,0.819167,0.666944,0.758611,0.614167


fitting models:  60%|██████    | 6/10 [02:25<01:33, 23.39s/it]

: 